In [ ]:
import requests
import pandas as pd
import plotly.graph_objects as go
import streamlit as st

# -----------------------------------------------------------------------------
# 1. CONFIGURACIÓN Y ENTRADAS (Integración con Streamlit)
# -----------------------------------------------------------------------------

# Obtenemos las variables del contexto de app.py
estado = locals().get("ESTADO_SELECCIONADO", "Aguascalientes")
palette = locals().get("active_palette", ["#0576F3", "#36F48C", "#F47806", "#F479F4", "#F3F40B"])

# Token y API INEGI
TOKEN = "460ccba7-40b7-08a2-47dd-7301e6b6fbbc"
BASE_URL = "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR"

# Diccionario de Estados para mapear Nombre -> Código INEGI
ESTADOS_INEGI = {
    "Aguascalientes": "01", "Baja California": "02", "Baja California Sur": "03", "Campeche": "04",
    "Coahuila": "05", "Colima": "06", "Chiapas": "07", "Chihuahua": "08", "Ciudad de México": "09",
    "Durango": "10", "Guanajuato": "11", "Guerrero": "12", "Hidalgo": "13", "Jalisco": "14",
    "México": "15", "Michoacán": "16", "Morelos": "17", "Nayarit": "18", "Nuevo León": "19",
    "Oaxaca": "20", "Puebla": "21", "Querétaro": "22", "Quintana Roo": "23", "San Luis Potosí": "24",
    "Sinaloa": "25", "Sonora": "26", "Tabasco": "27", "Tamaulipas": "28", "Tlaxcala": "29",
    "Veracruz": "30", "Yucatán": "31", "Zacatecas": "32"
}

# Validación simple
if estado not in ESTADOS_INEGI:
    st.error(f"El estado '{estado}' no se encuentra en el catálogo.")
    st.stop()

codigo_estado = ESTADOS_INEGI[estado]

# Función para obtener colores de tu paleta cíclicamente
def get_color(idx):
    return palette[idx % len(palette)]

# Función robusta de petición a API
def fetch_inegi_data(indicators, area_code):
    """
    Realiza la petición a la API del INEGI para una lista de indicadores.
    Devuelve el JSON parseado o None si falla.
    """
    ids_str = ",".join(indicators)
    # URL estándar de la API
    url = f"{BASE_URL}/{ids_str}/es/{area_code}/false/BISE/2.0/{TOKEN}?type=json"

    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        # Intentamos decodificar JSON
        return response.json()
    except requests.exceptions.RequestException as e:
        # Errores de conexión silenciosos en consola, visibles si es crítico
        return None
    except ValueError:
        # Error al parsear JSON (el error que tenías)
        return None

# -----------------------------------------------------------------------------
# 2. GRÁFICA 1: CRECIMIENTO HISTÓRICO Y POBLACIÓN (Población Total)
# -----------------------------------------------------------------------------
st.markdown("### 📈 Crecimiento histórico y población total")

data_growth = fetch_inegi_data(["1002000001"], codigo_estado)

if data_growth and 'Series' in data_growth:
    # Lista temporal para procesar los datos antes de separar en X e Y
    raw_data = []

    # Extraer datos de la primera serie
    series = data_growth['Series'][0] if data_growth['Series'] else {}
    obs_list = series.get('OBSERVATIONS', [])

    for obs in obs_list:
        try:
            # Convertimos el año a entero para poder filtrar y ordenar
            anio = int(obs.get('TIME_PERIOD'))
            valor = float(obs.get('OBS_VALUE', 0))
            
            # FILTRO: Solo años desde 1990 en adelante
            if anio >= 1990:
                raw_data.append({"anio": anio, "valor": valor})
        except ValueError:
            continue

    # ORDENAMIENTO: Ordenar por año de menor a mayor (Ascendente)
    raw_data.sort(key=lambda x: x["anio"])

    # Separar en listas para Plotly
    fechas = [str(item["anio"]) for item in raw_data]
    valores = [item["valor"] for item in raw_data]

    if valores:
        # Calcular tasas de crecimiento
        crecimiento = [None] # El primer valor (1990) no tiene previo para comparar
        for i in range(1, len(valores)):
            prev = valores[i-1]
            curr = valores[i]
            # Fórmula de tasa de crecimiento
            tasa = ((curr - prev) / prev) * 100 if prev != 0 else 0
            crecimiento.append(round(tasa, 2))

        fig1 = go.Figure()

        # Barras: Población
        fig1.add_trace(go.Bar(
            x=fechas, y=valores,
            name="Población",
            text=[f"{v:,.0f}" for v in valores],
            textposition="outside",
            marker_color=get_color(0)
        ))

        # Línea: Crecimiento
        fig1.add_trace(go.Scatter(
            x=fechas, y=crecimiento,
            name="Crecimiento Anual (%)",
            yaxis="y2",
            mode="lines+markers",
            line=dict(color=get_color(1), width=3)
        ))

        fig1.update_layout(
            title=dict(text=f"Histórico: {estado} (1990-2020)", xanchor="center", x=0.5),
            xaxis=dict(title="Año"),
            yaxis=dict(title="Población", side="left", tickformat=","),
            yaxis2=dict(
                title="Crecimiento %",
                overlaying="y",
                side="right",
                showgrid=False
            ),
            legend=dict(orientation="h", y=-0.2, x=0.5, xanchor="center"),
            template="plotly_white",
            # === MODIFICACIÓN 1: Aumentamos margen inferior (b) a 150 ===
            margin=dict(t=50, b=150, l=50, r=50) 
        )

        # === MODIFICACIÓN 2: Agregar leyenda de fuente ===
        fig1.add_annotation(
            text="Fuente: INEGI",
            xref="paper", yref="paper",
            x=0, y=-0.3,  # Posición inferior izquierda
            showarrow=False,
            xanchor='left', yanchor='top',
            font=dict(size=12, color="gray")
        )

        st.plotly_chart(fig1, use_container_width=True)

        # ---------------------------------------------------
        # TABLA DE DATOS 1 (Crecimiento)
        # ---------------------------------------------------
        st.markdown("**Datos detallados (Crecimiento)**")
        
        # Crear DataFrame para la tabla
        df_table1 = pd.DataFrame({
            "Año": fechas,
            "Población": valores,
            "Crecimiento (%)": [c if c is not None else 0 for c in crecimiento]
        })
        # Ordenar descendente (más reciente arriba)
        df_table1 = df_table1.sort_values("Año", ascending=False)

        st.dataframe(
            df_table1,
            use_container_width=True,
            hide_index=True,
            column_config={
                "Año": st.column_config.TextColumn("Año"),
                "Población": st.column_config.NumberColumn(format="%,d"),
                "Crecimiento (%)": st.column_config.NumberColumn(format="%.2f%%")
            }
        )

    else:
        st.warning("Datos insuficientes para graficar población histórica (Post-1990).")
else:
    st.error("No se pudo obtener la información de Crecimiento Poblacional.")


# -----------------------------------------------------------------------------
# 3. GRÁFICA 2: POBLACIÓN POR GÉNERO
# -----------------------------------------------------------------------------
st.markdown("### 👥 Distribución por género")

# 1002000002: Hombres, 1002000003: Mujeres
data_sex = fetch_inegi_data(["1002000002", "1002000003"], codigo_estado)

if data_sex and 'Series' in data_sex:
    records = []
    for serie in data_sex['Series']:
        indicador_id = serie.get('INDICADOR')
        label = "Hombres" if indicador_id == "1002000002" else "Mujeres"

        for obs in serie.get('OBSERVATIONS', []):
            records.append({
                "anio": obs.get('TIME_PERIOD'),
                "sexo": label,
                "valor": float(obs.get('OBS_VALUE', 0))
            })

    if records:
        df_sex = pd.DataFrame(records)
        df_pivot = df_sex.pivot_table(index="anio", columns="sexo", values="valor", aggfunc="sum").sort_index()

        fig2 = go.Figure()

        # Trazas
        cols = df_pivot.columns.tolist()
        for i, col_name in enumerate(cols):
            vals = df_pivot[col_name]
            fig2.add_trace(go.Bar(
                x=df_pivot.index, y=vals,
                name=col_name,
                marker_color=get_color(i),
                text=[f"{v:,.0f}" for v in vals],
                textposition="inside"
            ))

        fig2.update_layout(
            barmode="stack",
            title=dict(text=f"Población Hombres vs Mujeres - {estado}", xanchor="center", x=0.5),
            xaxis=dict(title="Año"),
            yaxis=dict(title="Población", tickformat=","),
            legend=dict(orientation="h", y=-0.2, x=0.5, xanchor="center"),
            template="plotly_white",
            # === MODIFICACIÓN 1: Agregar margen inferior ===
            margin=dict(b=150)
        )

        # === MODIFICACIÓN 2: Agregar leyenda de fuente ===
        fig2.add_annotation(
            text="Fuente: INEGI",
            xref="paper", yref="paper",
            x=0, y=-0.3,
            showarrow=False,
            xanchor='left', yanchor='top',
            font=dict(size=12, color="gray")
        )

        st.plotly_chart(fig2, use_container_width=True)

        # ---------------------------------------------------
        # TABLA DE DATOS 2 (Género)
        # ---------------------------------------------------
        st.markdown("**Datos detallados (Género)**")
        
        # Preparamos el DF para la tabla (Reset index para que 'anio' sea columna)
        df_table2 = df_pivot.reset_index().sort_values("anio", ascending=False)
        
        st.dataframe(
            df_table2,
            use_container_width=True,
            hide_index=True,
            column_config={
                "anio": st.column_config.TextColumn("Año"),
                "Hombres": st.column_config.NumberColumn(format="%,d"),
                "Mujeres": st.column_config.NumberColumn(format="%,d")
            }
        )

# -----------------------------------------------------------------------------
# 4. GRÁFICA 3: POBLACIÓN POR GRUPOS DE EDAD
# -----------------------------------------------------------------------------
st.markdown("### 🎂 Distribución por grupos de edad")

# Lista completa de indicadores de edad
IDS_EDAD = [
    "1002000058","1002000061","1002000067","1002000070","1002000073","1002000076",
    "1002000079","1002000082","1002000085","1002000088","1002000091",
    "1002000094","1002000097","1002000100","1002000103","1002000106","1002000109",
    "1002000112","1002000115","1002000118","1002000121","1002000124"
]

# Mapeo de IDs a Grupos
MAP_EDAD = {
    "1002000058":"0-19", "1002000061":"0-19", "1002000067":"0-19", "1002000070":"0-19",
    "1002000073":"20-64", "1002000076":"20-64", "1002000079":"20-64", "1002000082":"20-64",
    "1002000085":"20-64", "1002000088":"20-64", "1002000091":"20-64", "1002000094":"20-64",
    "1002000097":"20-64"
    # El resto caerá en "65+ años" por defecto en la lógica de abajo
}

series_acumuladas = []
bar = st.progress(0, text="Cargando datos demográficos...")

total_ids = len(IDS_EDAD)
for idx, id_ind in enumerate(IDS_EDAD):
    resp = fetch_inegi_data([id_ind], codigo_estado)
    if resp and 'Series' in resp:
        series_acumuladas.extend(resp['Series'])
    bar.progress((idx + 1) / total_ids)

bar.empty()

if series_acumuladas:
    age_records = []
    for s in series_acumuladas:
        iid = s.get('INDICADOR')
        if iid == "1002000124":
            grupo = "No especificado"
        else:
            grupo = MAP_EDAD.get(iid, "65+ años")

        for obs in s.get('OBSERVATIONS', []):
            try:
                val = float(obs.get('OBS_VALUE', 0))
                age_records.append({
                    "anio": obs.get('TIME_PERIOD'),
                    "grupo": grupo,
                    "valor": val
                })
            except:
                continue

    if age_records:
        df_age = pd.DataFrame(age_records)
        df_agrupado = df_age.groupby(["anio", "grupo"])["valor"].sum().reset_index()
        df_pivot_age = df_agrupado.pivot(index="anio", columns="grupo", values="valor").fillna(0)

        fig3 = go.Figure()

        grupos_orden = ["0-19", "20-64", "65+ años", "No especificado"]

        idx_color = 0
        for g in grupos_orden:
            if g in df_pivot_age.columns:
                vals = df_pivot_age[g]
                fig3.add_trace(go.Bar(
                    x=df_pivot_age.index,
                    y=vals,
                    name=g,
                    marker_color=get_color(idx_color),
                    text=[f"{v:,.0f}" for v in vals],
                    textposition="inside" if idx_color < 2 else "outside"
                ))
                idx_color += 1

        fig3.update_layout(
            barmode="stack",
            title=dict(text=f"Evolución por Grandes Grupos de Edad - {estado}", xanchor="center", x=0.5),
            xaxis=dict(title="Año"),
            yaxis=dict(title="Población", tickformat=","),
            legend=dict(orientation="h", y=-0.2, x=0.5, xanchor="center"),
            template="plotly_white",
            # === MODIFICACIÓN 1: Agregar margen inferior ===
            margin=dict(b=150)
        )

        # === MODIFICACIÓN 2: Agregar leyenda de fuente ===
        fig3.add_annotation(
            text="Fuente: INEGI",
            xref="paper", yref="paper",
            x=0, y=-0.3,
            showarrow=False,
            xanchor='left', yanchor='top',
            font=dict(size=12, color="gray")
        )

        st.plotly_chart(fig3, use_container_width=True)

        # ---------------------------------------------------
        # TABLA DE DATOS 3 (Grupos de Edad)
        # ---------------------------------------------------
        st.markdown("**Datos detallados (Grupos de Edad)**")
        
        # Preparamos el DF para la tabla
        df_table3 = df_pivot_age.reset_index().sort_values("anio", ascending=False)
        
        # Configuración de columnas dinámica (para aplicar formato a todas las columnas de grupos)
        cols_config = {"anio": st.column_config.TextColumn("Año")}
        for col in grupos_orden:
            if col in df_table3.columns:
                cols_config[col] = st.column_config.NumberColumn(format="%,d")

        st.dataframe(
            df_table3,
            use_container_width=True,
            hide_index=True,
            column_config=cols_config
        )

    else:
        st.warning("No se encontraron datos detallados de edad.")
else:
    st.error("Error al obtener indicadores de edad.")